In [1]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [2]:
!pip install newspaper3k


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 46.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=81417bd102f7147f0cf509be3ed5048afbb5f88af6449c5529489c30e1570725
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=09c18ed5393cf5063d2982c1735bf8b42629ac072a2bc8e2e1cbd8dd2bf27665
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1b

In [3]:
!wget -q -O - ipv4.icanhazip.com

35.221.33.233


In [8]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from newspaper import Article

# Model and tokenizer
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Setting the page title
st.title("Financial Sentiment Analysis")

# Input option: Text or URL
input_option = st.radio("Choose input type:", ["Text Input", "URL Input"])

if input_option == "Text Input":
    text_input = st.text_area("Enter Financial News:", "DEMO : Tesla stock is soaring after record-breaking earnings.")
else:
    url_input = st.text_input("Enter URL to scrape headline:")
    if url_input:
        try:
            # Scrape the headline from the URL
            article = Article(url_input)
            article.download()
            article.parse()
            text_input = article.title  # Use the article's title as the headline
            st.success(f"Scraped Headline: {text_input}")
        except Exception as e:
            st.error(f"Failed to extract headline: {e}")
            text_input = ""

# Function to perform sentiment analysis
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    sentiment_class = outputs.logits.argmax(dim=1).item()
    sentiment_mapping = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    predicted_sentiment = sentiment_mapping.get(sentiment_class, 'Unknown')
    return predicted_sentiment, outputs.logits.softmax(dim=1)[0].tolist()

# Button to trigger sentiment analysis
if st.button("Analyze Sentiment"):
    # Checking if the input text is not empty
    if text_input and text_input.strip():
        # Showing loading spinner while processing
        with st.spinner("Analyzing sentiment..."):
            sentiment, confidence_scores = predict_sentiment(text_input)

            # Considering a threshold for sentiment prediction
            threshold = 0.5

            # Changing the success message background color based on sentiment and threshold
            if sentiment == 'Positive' and confidence_scores[2] > threshold:
                st.success(f"Sentiment: {sentiment} (Confidence: {confidence_scores[2]:.3f})")
            elif sentiment == 'Negative' and confidence_scores[0] > threshold:
                st.error(f"Sentiment: {sentiment} (Confidence: {confidence_scores[0]:.3f})")
            elif sentiment == 'Neutral' and confidence_scores[1] > threshold:
                st.info(f"Sentiment: {sentiment} (Confidence: {confidence_scores[1]:.3f})")
            else:
                st.warning("Low confidence, or sentiment not above threshold. Please try again.")
    else:
        st.warning("Please enter some valid text for sentiment analysis.")

# Optional: Displaying the raw sentiment scores
if st.checkbox("Show Raw Sentiment Scores"):
    if text_input and text_input.strip():
        _, raw_scores = predict_sentiment(text_input)
        st.info(f"Raw Sentiment Scores: \n Negative : {raw_scores[0]} \n Positive : {raw_scores[2]} \n Neutral : {raw_scores[1]}")

# footer
st.markdown(
    """
** Built and maintained by Swayam Mohanty **
    """
)


Overwriting app.py


In [7]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.221.33.233:8501

npx: installed 22 in 2.472s
your url is: https://some-peaches-rest.loca.lt
  Stopping...
^C
